<a href="https://colab.research.google.com/github/daliaezzat/ASOC_paper_code/blob/main/Phase_2_Optimization_phase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyDrive

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id': '1cA_rOXord2kQgzyxCNGfJi4CYOdDvG7F'})
download.GetContentFile('breast_cancer.zip')

In [ ]:
import zipfile
import io
data = zipfile.ZipFile('breast_cancer.zip', 'r')
data.extractall()

In [ ]:
train_dir = '/content/ratio_IDC3_breasr_cancer/train'
validation_dir = '/content/ratio_IDC3_breasr_cancer/validation'
test_dir = '/content/ratio_IDC3_breasr_cancer/test'

In [ ]:
download = drive.CreateFile({'id': '1C_9d-reZM-Xn1yw3R-d7NhWqvqG5edzg'})
download.GetContentFile('root.py')

In [ ]:
download = drive.CreateFile({'id': '1RfyAWotNIVC3xXAaGtOkJhmu_1a0A5N9'})
download.GetContentFile('SMA.py')


In [ ]:
def F1(DR):
    import numpy as np
    print("droupout_rate=")
    DR=round(min(DR),4)
    print(DR)
    image_size=280
    BS=32
    #----------------------------------------data preprocessing ---------------------------------------------
    from keras.preprocessing.image import ImageDataGenerator
    from keras.callbacks import ModelCheckpoint,EarlyStopping
    from sklearn.utils import class_weight

    train_datagen=ImageDataGenerator(rescale = 1./255.,
                                 rotation_range = 10,
                                 width_shift_range = 0.2,
                                 height_shift_range = 0.2,
                                 shear_range = 0.3,
                                 zoom_range = 0.25,
                                 horizontal_flip = True,
                                fill_mode='nearest')

    test_datagen=ImageDataGenerator(rescale=1./255)
    train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(image_size,image_size),
                                                  batch_size=BS,
                                                  class_mode='categorical')

    validation_generator=test_datagen.flow_from_directory(validation_dir,
                                                      target_size=(image_size,image_size),
                                                      batch_size=BS,
                                                      class_mode='categorical')

    #----------------------------------------------model------------------------------------------------------
    from tensorflow.keras import layers,optimizers
    from tensorflow.keras import models
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.regularizers import l2
    from tensorflow.keras.layers import Activation
    from sklearn.utils import compute_class_weight
    import numpy as np

    conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(image_size,image_size, 3))
    conv_base.summary()

    for i, layer in enumerate(conv_base.layers):
      print(i, layer.name)

    conv_base.trainable=False
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(100,activation='relu')) #kant l2=0.00001
    model.add(layers.Dropout(DR))
    model.add(layers.Dense(2,activation='softmax'))

    checkpointer = ModelCheckpoint(filepath='pretrained.weights.best.hdf5',monitor='val_loss', verbose = 1, save_best_only=True)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto',restore_best_weights=True)
    train_classes=train_generator.classes
    class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_classes),
                                        y = train_classes)
    class_weights = dict(zip(np.unique(train_classes), class_weights))
    model.compile(optimizer=optimizers.Adam(lr=2e-5),loss='categorical_crossentropy',metrics=['accuracy'])
    history=model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples/train_generator.batch_size,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples/train_generator.batch_size,
                    verbose=1,
                    class_weight=class_weights,
                    callbacks=[checkpointer,early])

    val_loss=history.history['val_loss']
    print (round(min(val_loss),4))
    return (round(min(val_loss),4))

In [ ]:
from SMA import BaseSMA, OriginalSMA
from numpy import sum, pi, exp, sqrt, cos


def func_sum(solution):
    return sum(solution ** 2)
lb = [0.1]
ub = [0.9]
problem_size = 1
obj_func = F1
verbose = True
epoch = 10
pop_size = 15
md1 = BaseSMA(obj_func, lb, ub, problem_size, verbose, epoch, pop_size)
best_pos1, best_fit1, list_loss1 = md1.train()
print(md1.solution[0])
print(md1.solution[1])
print(md1.loss_train)
md2 = OriginalSMA(obj_func, lb, ub, problem_size, verbose, epoch, pop_size)
best_pos2, best_fit2, list_loss2 = md2.train()
print(best_pos2)
print(best_fit2)
print(list_loss2)